In [ ]:
# Importar bibliotecas

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, r2_score

### Carregar Dados

In [ ]:
# Carregar os dados
df_costs = pd.read_csv('../datasets/healthcosts.csv')

In [ ]:
# Mostrar as primeiras linhas do dataframe
df_costs.head(5)

In [ ]:
# Mostrar as últimas linhas do dataframe
df_costs.tail(5)

In [ ]:
# Mostrar estrutura do dataframe
df_costs.info()

### Feature Engineering

In [ ]:
# Mostrar as colunas categóricas que possuem somente um valor possível
for column in df_costs.select_dtypes(include=['object']).columns:
  if df_costs[column].nunique() == 1:
    print(f'Coluna {column} possui somente um valor possível: {df_costs[column].unique()}')

In [ ]:
# Mostrar os valores possíveis para todas as colunas categóricas
for column in df_costs.select_dtypes(include=['object']).columns:
  print(f'Coluna {column} possui estes valores: {df_costs[column].unique()}')

In [ ]:
# Mostrar o percentual de valores ausentes para as colunas categóricas
for column in df_costs.select_dtypes(include=['object']).columns:
  contagem_nulas = df_costs[column].isnull().sum()
  print(f'{column}: {contagem_nulas / len(df_costs) * 100:.2f}%')

In [ ]:
# Apresentar Estatísticas Descritivas
df_costs.describe()

In [ ]:
# Mostrar as colunas numéricas que possuem somente um valor possível
for column in df_costs.select_dtypes(include=['number']).columns:
  if df_costs[column].nunique() == 1:
    print(f'Coluna {column} possui somente um valor possível: {df_costs[column].unique()}')

In [ ]:
# Mostrar o percentual de valores ausentes para as colunas numéricas
for column in df_costs.select_dtypes(include=['number']).columns:
  contagem_nulas = df_costs[column].isnull().sum()
  print(f'{column}: {contagem_nulas / len(df_costs) * 100:.2f}%')

In [ ]:
# Converter colunas categóricas com valores Yes e No para 1 e 0
for column in df_costs.select_dtypes(include=['object']).columns:
  valores_unicos = df_costs[column].unique()
  if set(valores_unicos).issubset(set(['yes', 'no'])):
    df_costs[column] = df_costs[column].apply(lambda x: 1 if x == 'yes' else 0)

### EDA

In [ ]:
# Mostrar distribuição de custos médicos
fig = px.histogram(
  df_costs,
  x='medical charges',
  nbins=30,
  title='Distribuição de Custos Médicos'
)

fig.show()

In [ ]:
# Mostrar distribuição de idade
fig = px.histogram(
  df_costs,
  x='age',
  nbins=30,
  title='Distribuição da Idade'
)

fig.show()

In [ ]:
# Mostrar a quantidade de filhos
fig = px.histogram(
  df_costs,
  x='children',
  title='Distribuição da Quantidade de Filhos'
)

fig.show()

In [ ]:
# Mostrar distribuição de BMI
fig = px.histogram(
  df_costs,
  x='bmi',
  nbins=30,
  title='Distribuição do BMI'
)

fig.show()

In [ ]:
# Mostrar a distribuição do gênero
fig = px.bar(
  df_costs['sex'].value_counts(),
  title='Distribuição por Gênero'
)

fig.show()

In [ ]:
# Mostrar a distribuição da variável Smoker
fig = px.bar(
  df_costs['smoker'].value_counts(),
  title='Distribuição de Fumante'
)

fig.show()

In [ ]:
# Mostrar a distribuição de região
fig = px.bar(
  df_costs['region'].value_counts(),
  title='Distribuição por Região'
)

fig.show()

In [ ]:
# Boxplot de custos médicos por idade
fig = px.box(
  df_costs,
  x='age',
  y='medical charges',
  title='Boxplot de Custos Médicos por Idade'
)

fig.show()

In [ ]:
# Boxplot de custos médicos por gênero
fig = px.box(
  df_costs,
  x='sex',
  y='medical charges',
  title='Boxplot de Custos Médicos por Gênero'
)

fig.show()

In [ ]:
# Boxplot de custos médicos por Smoker (Fumante)
fig = px.box(
  df_costs,
  x='smoker',
  y='medical charges',
  title='Boxplot de Custos Médicos por Status de Fumante'
)

fig.show()

In [ ]:
# Boxplot de Custos Médicos por Região
fig = px.box(
  df_costs,
  x='region',
  y='medical charges',
  title='Boxplot de Custos Médicos por Região'
)

fig.show()

In [ ]:
# Plot de correlação das variáveis numéricas
corr_matrix = df_costs.select_dtypes(include=['number']).corr()

In [ ]:
# Mostrar a matriz de correlação
corr_matrix

In [ ]:
fig = go.Figure()

fig.add_trace(
  go.Heatmap(
    x=corr_matrix.columns,
    y=corr_matrix.index,
    z=np.array(corr_matrix),
    text=corr_matrix.values,
    texttemplate='%{text:.3f}',
    colorscale=px.colors.diverging.RdBu,
    zmin=-1,
    zmax=1
  )
)

fig.show()

### Preparação dos dados

In [ ]:
# Preparar dados para o modelo
X = df_costs.drop(columns=['medical charges'])
y = df_costs['medical charges']

In [ ]:
# Column Transformer para normalizar variáveis numéricas e OnHotEncode para categóricas
numeric_features = X.select_dtypes(include=['number']).columns
categorical_features = X.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
  transformers=[
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
  ]
)

In [ ]:
# Dividir os dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(
  X,
  y,
  test_size=0.2,
  random_state=51
)

In [ ]:
# Aplicar o Column Transformer nos dados de treinamento e teste
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [ ]:
# Mostrar as dimensões dos conjuntos
print(f'Dados de treinamento: {X_train.shape}')
print(f'Dados de teste: {X_test.shape}')

### Treinamento do modelo

In [ ]:
# Criar o modelo de Bagging Regressor
bagging_model = BaggingRegressor(
  estimator=LinearRegression(),
  n_estimators=5,
  random_state=51,
)

In [ ]:
# Treinar o modelo
bagging_model.fit(X_train, y_train)

### Análise dos Resultados

In [ ]:
# Fazer predições com base no modelo treinado
y_pred = bagging_model.predict(X_test)

In [ ]:
# Mostrar y_pred
y_pred

In [ ]:
# Avaliar métricas do modelo
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [ ]:
# Mostrar o Erro e R2 do Modelo
print(f'Root Mean Squared Error: {rmse}')
print(f'R2: {r2}')

In [ ]:
# Calcular a importância das features usando os coeficientes

# Obter os coeficiente de cada estimador
coefs = np.array([estimator.coef_ for estimator in bagging_model.estimators_])

# Calcular a média dos coeficientes absolutos
feature_importance = np.mean(np.abs(coefs), axis=0)

# Normalizar as importâncias
feature_importance = feature_importance / np.sum(feature_importance)

In [ ]:
# Obter os nomes das features
feature_names = preprocessor.get_feature_names_out()

In [ ]:
# Criar um dataframe com as importâncias e os nomes
importance_df = pd.DataFrame(
  {
    'feature': feature_names,
    'importance': feature_importance
  }
)

# Ordenar o Dataframe pela importância
importance_df = importance_df.sort_values('importance', ascending=True)

In [ ]:
# Criar o gráfico de barras para mostrar a importância das features
fig = px.bar(
  importance_df,
  x='importance',
  y='feature',
  title='Importância das Features',
  orientation='h'
)

fig.update_xaxes(tickangle=45)
fig.show()

### Verificar propriedades do modelo

In [ ]:
bagging_model.estimators_samples_

In [ ]:
bagging_model.estimators_samples_[2].shape

In [ ]:
bagging_model.estimators_features_

### Salvar dados e pre-processador do modelo

In [ ]:
# Salvar dataframe como CSV
df_costs.to_csv('../datasets/healthcosts_cleaned.csv', index=False)

In [ ]:
# Salvar o preprocessor
import joblib

joblib.dump(preprocessor, './preprocessor_dataset_healthcosts.pkl')